In [253]:
# import module
import requests
from bs4 import BeautifulSoup

In [1]:
HEADERS = {'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
        AppleWebKit/537.36 (KHTML, like Gecko) \
        Chrome/90.0.4430.212 Safari/537.36'}

COOKIES = {}

In [255]:
def get_data(url:str):
    
    page = requests.get(url, cookies=COOKIES, headers=HEADERS)
    return page

In [257]:
def customer_review(soup):
    data_str = ""
  
    for item in soup.find_all("span", class_="a-size-base review-text review-text-content"):
        data_str = data_str + item.get_text()
  
    result = data_str.split("\n")
    return (result)
  

In [258]:
def asin_number(soup):
    data_asins = []
    
    for item in soup.find_all("div", {"data-component-type": "s-search-result"}):
        data_asins.append(item['data-asin'])
    return data_asins

In [259]:
def fetch_href(soup):
    links = []
  
    for item in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        links.append(item['href'])
    
    return links[0]

In [260]:
url = "https://www.amazon.in/s?k=boat+earphones"

In [261]:
response=get_data(url)
soup=BeautifulSoup(response.content)

In [262]:
data_asins = asin_number(soup)

In [263]:
all_reviews = []
review_data = []

In [264]:
data_asins

['B07S9S86BF',
 'B097DTJRWZ',
 'B071Z8M4KX',
 'B07S9S86BF',
 'B07GQD4K6L',
 'B01MF8MB65',
 'B01M9C51T9',
 'B07S8PSW59',
 'B07KY3FNQP',
 'B07234XXJF',
 'B08W9BQ4G8',
 'B092KC9YT6',
 'B08H9Z3XQW',
 'B08MD2CTDB',
 'B07C2VJXP4',
 'B07KXNS51J',
 'B08JQN8DGZ',
 'B08GG6GQ5B',
 'B08TTXNZ4Y',
 'B07Q95M9QZ',
 'B09R9PPKTF',
 'B09DD7F3CB']

In [ ]:
for data_asin in data_asins:
    url = f"https://www.amazon.in/dp/{data_asin}"
    
    response=get_data(url)
    soup=BeautifulSoup(response.content)
    link = fetch_href(soup)
    
    i = 0
    while 1:
        i += 1
        url = f"https://www.amazon.in{link}&pageNumber=1"
        response=get_data(url)
        soup=BeautifulSoup(response.text)
        review_data = customer_review(soup)
        review_data = [review for review in review_data if len(review) > 0]
        if len(review_data) == 0:
            break
        
        all_reviews += review_data
        print(f"Total reviews: {len(all_reviews)}")